# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846855915947                   -0.70    4.5         
  2   -7.852313886691       -2.26       -1.53    1.0   24.5ms
  3   -7.852615418873       -3.52       -2.56    1.5   26.9ms
  4   -7.852645925983       -4.52       -2.88    2.5   33.7ms
  5   -7.852646496018       -6.24       -3.18    1.0   24.5ms
  6   -7.852646677447       -6.74       -3.98    1.0   24.5ms
  7   -7.852646686116       -8.06       -5.14    1.5   27.6ms
  8   -7.852646686722       -9.22       -5.40    2.5   33.5ms
  9   -7.852646686728      -11.20       -5.74    1.0   24.8ms
 10   -7.852646686730      -11.85       -6.37    1.0   24.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846791917172                   -0.70           4.8         
  2   -7.852525085510       -2.24       -1.64   0.80    2.2    268ms
  3   -7.852635557331       -3.96       -2.74   0.80    1.0   22.7ms
  4   -7.852646503373       -4.96       -3.23   0.80    2.2   30.3ms
  5   -7.852646672951       -6.77       -4.06   0.80    1.2   24.6ms
  6   -7.852646686309       -7.87       -4.79   0.80    1.8   26.9ms
  7   -7.852646686722       -9.38       -5.58   0.80    1.8   27.0ms
  8   -7.852646686729      -11.13       -6.99   0.80    2.0   27.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.355891e+01     3.936679e+00
 * time: 0.4190831184387207
     1     1.018362e+00     1.892724e+00
 * time: 0.6144440174102783
     2    -1.671756e+00     2.251118e+00
 * time: 0.6377520561218262
     3    -3.585475e+00     2.122780e+00
 * time: 0.6713149547576904
     4    -4.720523e+00     1.866894e+00
 * time: 0.7050330638885498
     5    -6.389947e+00     1.228906e+00
 * time: 0.7387411594390869
     6    -7.217424e+00     7.549511e-01
 * time: 0.7721090316772461
     7    -7.438192e+00     7.453749e-01
 * time: 0.7952060699462891
     8    -7.720323e+00     2.552353e-01
 * time: 0.8187000751495361
     9    -7.784678e+00     9.445510e-02
 * time: 0.8420829772949219
    10    -7.819143e+00     7.925538e-02
 * time: 0.865286111831665
    11    -7.835908e+00     6.459594e-02
 * time: 0.8885729312896729
    12    -7.841652e+00     4.031123e-02
 * time: 0.9117910861968994
    13    -7.844492e+00     2.582869e-02
 * time: 0.93516898155

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846878243737                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645911170                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.64s
  3   -7.852646686730      -13.30       -7.25    168ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.4223897760304476e-7
|ρ_newton - ρ_scfv| = 3.2598536113650966e-7
|ρ_newton - ρ_dm|   = 0.011554306987132326
